In [37]:
import pandas as pd
import re

# Load the CSV files
df1 = pd.read_csv(r'C:\Users\Admin.DESKTOP-M4R2VLU\week7\data\CheMed123.csv')
df2 = pd.read_csv(r'C:\Users\Admin.DESKTOP-M4R2VLU\week7\data\DoctorsET.csv')
df3 = pd.read_csv(r'C:\Users\Admin.DESKTOP-M4R2VLU\week7\data\lobelia4cosmetics.csv')

# Merge the DataFrames
merged_df = pd.concat([df1, df2, df3], ignore_index=True)

In [38]:
merged_df

,Channel Title,Channel Username,ID,Message,Date,Media Path,Unnamed: 0
0,CheMed,@CheMed123,97,"⚠️Notice!\nDear esteemed customers,\nDue to fo...",2023-02-10 12:23:06+00:00,photos\@CheMed123_97.jpg,NaN
1,CheMed,@CheMed123,96,Mela-One በውስጡ ሆርሞን ያለው ድንገተኛ ወሊድ መቆጣጠርያ ሲሆን ያለ...,2023-02-02 08:58:52+00:00,photos\@CheMed123_96.jpg,NaN
2,CheMed,@CheMed123,95,አዚትሮማይሲን በሃኪም መድሃኒት ማዘዣ ከሚታዘዙ አንቲባዮቲኮች አንዱ ሲሆን...,2023-02-01 08:59:37+00:00,photos\@CheMed123_95.jpg,NaN
3,CheMed,@CheMed123,94,Che-Med Trivia #3\n\nምግብና መጠጦች አንዳንድ መድሃኒቶች በደ...,2023-01-31 09:19:53+00:00,photos\@CheMed123_94.jpg,NaN
4,CheMed,@CheMed123,93,"Che-Med Trivia #2\n\nእንደ Ciprofloxacin, Doxycy...",2023-01-30 09:45:25+00:00,photos\@CheMed123_93.jpg,NaN
...,...,...,...,...,...,...,...
742,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,11830,ENFAMIL AR 354GM \nPrice 4200 birr \nTelegram ...,2024-09-12 05:46:18+00:00,photos\@lobelia4cosmetics_11830.jpg,575.0
743,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,11829,ENFAMIL ENFANT FORMULA 352GM \nPrice 3800 birr...,2024-09-12 05:46:18+00:00,photos\@lobelia4cosmetics_11829.jpg,576.0
744,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,11828,Nido 1.8 kg \nPrice 4000 birr \nTelegram https...,2024-09-12 05:46:17+00:00,photos\@lobelia4cosmetics_11828.jpg,577.0
745,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,11827,ENFAMIL NEURO 565GM \nPrice 5500 birr \nTelegr...,2024-09-12 05:46:17+00:00,photos\@lobelia4cosmetics_11827.jpg,578.0


In [39]:
print(merged_df.dtypes)

Channel Title        object
Channel Username     object
ID                    int64
Message              object
Date                 object
Media Path           object
Unnamed: 0          float64
dtype: object


In [40]:
#merged_df['Unnamed: 0'] = pd.to_numeric(merged_df['Unnamed: 0'], errors='coerce')

# Remove duplicates
merged_df.drop_duplicates(inplace=True)

# Fill missing values (example: fill with the mean for numeric columns)
#merged_df.fillna(merged_df.mean(), inplace=True)

# Drop unnecessary columns (example: drop a column named 'unnecessary_column')
if 'Channel Title' in merged_df.columns:
    merged_df.drop(columns=['Channel Title'], inplace=True)
if  'Channel Username' in merged_df.columns:
    merged_df.drop(columns=['Channel Username'], inplace=True)
if  'Unnamed: 0' in merged_df.columns:
    merged_df.drop(columns=['Unnamed: 0'], inplace=True)
# Reset index if needed
merged_df.reset_index(drop=True, inplace=True)

In [41]:
merged_df

,ID,Message,Date,Media Path
0,97,"⚠️Notice!\nDear esteemed customers,\nDue to fo...",2023-02-10 12:23:06+00:00,photos\@CheMed123_97.jpg
1,96,Mela-One በውስጡ ሆርሞን ያለው ድንገተኛ ወሊድ መቆጣጠርያ ሲሆን ያለ...,2023-02-02 08:58:52+00:00,photos\@CheMed123_96.jpg
2,95,አዚትሮማይሲን በሃኪም መድሃኒት ማዘዣ ከሚታዘዙ አንቲባዮቲኮች አንዱ ሲሆን...,2023-02-01 08:59:37+00:00,photos\@CheMed123_95.jpg
3,94,Che-Med Trivia #3\n\nምግብና መጠጦች አንዳንድ መድሃኒቶች በደ...,2023-01-31 09:19:53+00:00,photos\@CheMed123_94.jpg
4,93,"Che-Med Trivia #2\n\nእንደ Ciprofloxacin, Doxycy...",2023-01-30 09:45:25+00:00,photos\@CheMed123_93.jpg
...,...,...,...,...
742,11830,ENFAMIL AR 354GM \nPrice 4200 birr \nTelegram ...,2024-09-12 05:46:18+00:00,photos\@lobelia4cosmetics_11830.jpg
743,11829,ENFAMIL ENFANT FORMULA 352GM \nPrice 3800 birr...,2024-09-12 05:46:18+00:00,photos\@lobelia4cosmetics_11829.jpg
744,11828,Nido 1.8 kg \nPrice 4000 birr \nTelegram https...,2024-09-12 05:46:17+00:00,photos\@lobelia4cosmetics_11828.jpg
745,11827,ENFAMIL NEURO 565GM \nPrice 5500 birr \nTelegr...,2024-09-12 05:46:17+00:00,photos\@lobelia4cosmetics_11827.jpg


In [44]:
df = pd.DataFrame(merged_df)

# Function to clean text
def clean_text(text):
    if isinstance(text, str):  # Check if the input is a string
        # Remove emojis
        text = re.sub(r'[^\w\s,.-]', '', text)  # Keeps word characters, whitespace, and some punctuation
        # Remove hashtag at the beginning
        text = re.sub(r'^#', '', text)
        # Remove newline characters
        text = text.replace('\n', ' ')
        return text.strip()
    return ''  # Return empty string for non-string inputs

# Apply the cleaning function to the desired column
df['cleaned_text'] = df['Message'].apply(clean_text)

# Display the cleaned DataFrame
print(df[['Message', 'cleaned_text']])

                                               Message  \
0    ⚠️Notice!\nDear esteemed customers,\nDue to fo...   
1    Mela-One በውስጡ ሆርሞን ያለው ድንገተኛ ወሊድ መቆጣጠርያ ሲሆን ያለ...   
2    አዚትሮማይሲን በሃኪም መድሃኒት ማዘዣ ከሚታዘዙ አንቲባዮቲኮች አንዱ ሲሆን...   
3    Che-Med Trivia #3\n\nምግብና መጠጦች አንዳንድ መድሃኒቶች በደ...   
4    Che-Med Trivia #2\n\nእንደ Ciprofloxacin, Doxycy...   
..                                                 ...   
742  ENFAMIL AR 354GM \nPrice 4200 birr \nTelegram ...   
743  ENFAMIL ENFANT FORMULA 352GM \nPrice 3800 birr...   
744  Nido 1.8 kg \nPrice 4000 birr \nTelegram https...   
745  ENFAMIL NEURO 565GM \nPrice 5500 birr \nTelegr...   
746  NIDO USA 2.2KG\nPrice 5900 birr \nTelegram htt...   

                                          cleaned_text  
0    Notice Dear esteemed customers, Due to four-da...  
1    Mela-One በውስጡ ሆርሞን ያለው ድንገተኛ ወሊድ መቆጣጠርያ ሲሆን ያለ...  
2    አዚትሮማይሲን በሃኪም መድሃኒት ማዘዣ ከሚታዘዙ አንቲባዮቲኮች አንዱ ሲሆን...  
3    Che-Med Trivia 3  ምግብና መጠጦች አንዳንድ መድሃኒቶች በደንብ ...  
4    Che-Med Trivi

In [46]:
# Define a function to remove emojis
def remove_emojis(text):
    emoji_pattern = re.compile(
        "[" 
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F700-\U0001F77F"  # alchemical symbols
        "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U0001FA00-\U0001FA6F"  # Chess Symbols
        "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251" 
        "]+", 
        flags=re.UNICODE
    )
    #return emoji_pattern.sub(r'', text)

# Apply the function to the 'Message' column
df['Message'] = df['Message'].apply(remove_emojis)

# Display the updated DataFrame
print(df.head())


   ID Message                       Date                Media Path  \
0  97    None  2023-02-10 12:23:06+00:00  photos\@CheMed123_97.jpg   
1  96    None  2023-02-02 08:58:52+00:00  photos\@CheMed123_96.jpg   
2  95    None  2023-02-01 08:59:37+00:00  photos\@CheMed123_95.jpg   
3  94    None  2023-01-31 09:19:53+00:00  photos\@CheMed123_94.jpg   
4  93    None  2023-01-30 09:45:25+00:00  photos\@CheMed123_93.jpg   

                                        cleaned_text  
0  Notice Dear esteemed customers, Due to four-da...  
1  Mela-One በውስጡ ሆርሞን ያለው ድንገተኛ ወሊድ መቆጣጠርያ ሲሆን ያለ...  
2  አዚትሮማይሲን በሃኪም መድሃኒት ማዘዣ ከሚታዘዙ አንቲባዮቲኮች አንዱ ሲሆን...  
3  Che-Med Trivia 3  ምግብና መጠጦች አንዳንድ መድሃኒቶች በደንብ ...  
4  Che-Med Trivia 2  እንደ Ciprofloxacin, Doxycycli...  


In [47]:
df

,ID,Message,Date,Media Path,cleaned_text
0,97,None,2023-02-10 12:23:06+00:00,photos\@CheMed123_97.jpg,"Notice Dear esteemed customers, Due to four-da..."
1,96,None,2023-02-02 08:58:52+00:00,photos\@CheMed123_96.jpg,Mela-One በውስጡ ሆርሞን ያለው ድንገተኛ ወሊድ መቆጣጠርያ ሲሆን ያለ...
2,95,None,2023-02-01 08:59:37+00:00,photos\@CheMed123_95.jpg,አዚትሮማይሲን በሃኪም መድሃኒት ማዘዣ ከሚታዘዙ አንቲባዮቲኮች አንዱ ሲሆን...
3,94,None,2023-01-31 09:19:53+00:00,photos\@CheMed123_94.jpg,Che-Med Trivia 3 ምግብና መጠጦች አንዳንድ መድሃኒቶች በደንብ ...
4,93,None,2023-01-30 09:45:25+00:00,photos\@CheMed123_93.jpg,"Che-Med Trivia 2 እንደ Ciprofloxacin, Doxycycli..."
...,...,...,...,...,...
742,11830,None,2024-09-12 05:46:18+00:00,photos\@lobelia4cosmetics_11830.jpg,ENFAMIL AR 354GM Price 4200 birr Telegram ht...
743,11829,None,2024-09-12 05:46:18+00:00,photos\@lobelia4cosmetics_11829.jpg,ENFAMIL ENFANT FORMULA 352GM Price 3800 birr ...
744,11828,None,2024-09-12 05:46:17+00:00,photos\@lobelia4cosmetics_11828.jpg,Nido 1.8 kg Price 4000 birr Telegram httpst....
745,11827,None,2024-09-12 05:46:17+00:00,photos\@lobelia4cosmetics_11827.jpg,ENFAMIL NEURO 565GM Price 5500 birr Telegram...


In [48]:
df.drop(columns=['Message'], inplace=True)

In [49]:
df

,ID,Date,Media Path,cleaned_text
0,97,2023-02-10 12:23:06+00:00,photos\@CheMed123_97.jpg,"Notice Dear esteemed customers, Due to four-da..."
1,96,2023-02-02 08:58:52+00:00,photos\@CheMed123_96.jpg,Mela-One በውስጡ ሆርሞን ያለው ድንገተኛ ወሊድ መቆጣጠርያ ሲሆን ያለ...
2,95,2023-02-01 08:59:37+00:00,photos\@CheMed123_95.jpg,አዚትሮማይሲን በሃኪም መድሃኒት ማዘዣ ከሚታዘዙ አንቲባዮቲኮች አንዱ ሲሆን...
3,94,2023-01-31 09:19:53+00:00,photos\@CheMed123_94.jpg,Che-Med Trivia 3 ምግብና መጠጦች አንዳንድ መድሃኒቶች በደንብ ...
4,93,2023-01-30 09:45:25+00:00,photos\@CheMed123_93.jpg,"Che-Med Trivia 2 እንደ Ciprofloxacin, Doxycycli..."
...,...,...,...,...
742,11830,2024-09-12 05:46:18+00:00,photos\@lobelia4cosmetics_11830.jpg,ENFAMIL AR 354GM Price 4200 birr Telegram ht...
743,11829,2024-09-12 05:46:18+00:00,photos\@lobelia4cosmetics_11829.jpg,ENFAMIL ENFANT FORMULA 352GM Price 3800 birr ...
744,11828,2024-09-12 05:46:17+00:00,photos\@lobelia4cosmetics_11828.jpg,Nido 1.8 kg Price 4000 birr Telegram httpst....
745,11827,2024-09-12 05:46:17+00:00,photos\@lobelia4cosmetics_11827.jpg,ENFAMIL NEURO 565GM Price 5500 birr Telegram...
